In [1]:
#generate spark session
from pyspark.sql import SparkSession
    
spark = (
    SparkSession
    .builder
    .appName("Rule based model incident response")
    .config("spark.streaming.stopGracefullyOnShutdown",True)
    # .config("spark.sql.shuffle.partitions", "8") 
    .master("local[*]")
    .getOrCreate()
    )

spark

In [2]:
streaming_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/home/jovyan/sparkTEST/dataset/KDDCup99.csv")
)

In [3]:
streaming_df.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- lnum_compromised: integer (nullable = true)
 |-- lroot_shell: integer (nullable = true)
 |-- lsu_attempted: integer (nullable = true)
 |-- lnum_root: integer (nullable = true)
 |-- lnum_file_creations: integer (nullable = true)
 |-- lnum_shells: integer (nullable = true)
 |-- lnum_access_files: integer (nullable = true)
 |-- lnum_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullabl

In [4]:
from pyspark.sql.functions import when, col

data = streaming_df.withColumn('prediction', 
    when(
        # Rule for DOS attacks: high number of connections, specific services, etc.
        (col('srv_count') > 100) | (col('count') > 200), 'DOS'
    ).when(
        # Rule for U2R attacks: presence of root shell, suspicious commands, etc.
        (col('lroot_shell') == 1) | (col('lnum_root') > 5), 'U2R'
    ).when(
        # Rule for R2L attacks: failed login attempts, unusual service access, etc.
        (col('logged_in') == 0) & (col('num_failed_logins') > 5) | (col('hot') > 30), 'R2L'
    ).when(
        # Rule for probe attacks: scanning activities, unusual access patterns, etc.
        (col('lnum_access_files') > 5) | (col('count') > 50) & (col('srv_diff_host_rate') > 0.5), 'probe'
    ).otherwise('normal')
)


In [5]:
# Show the results
data.select('prediction').show()

+----------+
|prediction|
+----------+
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
|    normal|
+----------+
only showing top 20 rows



In [6]:
data.groupBy('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|    normal|154229|
|       DOS|339438|
|       U2R|   351|
|     probe|     2|
+----------+------+



In [7]:
from pyspark.sql.functions import when, col, expr
# Evaluate the model
# Assuming 'label' column contains the actual class

# Calculate True Positives, False Positives, True Negatives, False Negatives for each class
metrics = data.groupBy('label', 'prediction').count()

# Calculate TP, FP, FN for each label
tp = metrics.filter(expr("label = prediction")).groupBy("label").sum("count").withColumnRenamed("sum(count)", "TP")
fp = metrics.filter(expr("label != prediction")).groupBy("prediction").sum("count").withColumnRenamed("sum(count)", "FP")
fn = metrics.filter(expr("label != prediction")).groupBy("label").sum("count").withColumnRenamed("sum(count)", "FN")

# Join to calculate metrics
results = tp.join(fp, tp.label == fp.prediction, "outer").join(fn, "label", "outer").fillna(0)

# Calculate precision, recall, and F1-score for each class
results = results.withColumn("precision", col("TP") / (col("TP") + col("FP")))
results = results.withColumn("recall", col("TP") / (col("TP") + col("FN")))
results = results.withColumn("f1_score", 2 * (col("precision") * col("recall")) / (col("precision") + col("recall")))

# Show the precision, recall, and F1 score for each class
results.select("label", "precision", "recall", "f1_score").show()

# Calculate overall accuracy
correct_predictions = data.filter(col('prediction') == col('label')).count()
total_data = data.count()
accuracy = correct_predictions / total_data

# Calculate the weighted averages of precision, recall, and F1-score
total_tp = results.agg({"TP": "sum"}).collect()[0][0]
overall_precision = results.withColumn("weighted_precision", col("TP") * col("precision")).agg({"weighted_precision": "sum"}).collect()[0][0] / total_tp
overall_recall = results.withColumn("weighted_recall", col("TP") * col("recall")).agg({"weighted_recall": "sum"}).collect()[0][0] / total_tp
overall_f1 = results.withColumn("weighted_f1", col("TP") * col("f1_score")).agg({"weighted_f1": "sum"}).collect()[0][0] / total_tp

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {overall_precision:.2f}")
print(f"Recall: {overall_recall:.2f}")
print(f"F1 Score: {overall_f1:.2f}")

# Optionally, show the confusion matrix
data.groupBy('label', 'prediction').count().show()

+---------------+------------------+------------------+------------------+
|          label|         precision|            recall|          f1_score|
+---------------+------------------+------------------+------------------+
|           null|               0.0|              null|              null|
|           null|               0.0|              null|              null|
|           null|               0.0|              null|              null|
|           back|              null|               0.0|              null|
|buffer_overflow|              null|               0.0|              null|
|      ftp_write|              null|               0.0|              null|
|   guess_passwd|              null|               0.0|              null|
|           imap|              null|               0.0|              null|
|        ipsweep|              null|               0.0|              null|
|           land|              null|               0.0|              null|
|     loadmodule|        